In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator(verbose=False).build(number)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count(), number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            desc="Creating dataset",
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(10000)
x_test, y_test = parallel_dataset_generation(1000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

In [5]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9947276761218599, 0.9956743600801629)

In [7]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.9959920884288929, 0.9970030325060855)

In [8]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe(max_rows=10)

In [9]:
y

,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,...,Name,NameSurname,SurnameName,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
0,ItalianFiscalCode,ItalianVAT,CadastreCode,Error,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,...,Name,NameSurname,SurnameName,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
1,ItalianFiscalCode,NaN,CadastreCode,Document,Tax,Plate,Address,NaN,ProvinceCode,NaN,...,Name,NameSurname,SurnameName,NaN,String,Error,PhoneNumber,NaN,BiologicalSex,Boolean
2,NaN,ItalianVAT,CadastreCode,Document,NaN,Plate,Address,ItalianZIPCode,NaN,Region,...,NaN,NaN,SurnameName,Surname,NaN,NaN,PhoneNumber,Date,Error,Boolean
3,NaN,ItalianVAT,CadastreCode,Document,Tax,NaN,Address,ItalianZIPCode,ProvinceCode,NaN,...,NaN,NameSurname,SurnameName,Surname,String,NaN,PhoneNumber,Date,BiologicalSex,Boolean
4,ItalianFiscalCode,ItalianVAT,NaN,NaN,NaN,NaN,Address,ItalianZIPCode,ProvinceCode,NaN,...,Name,NameSurname,SurnameName,Surname,String,NaN,PhoneNumber,Date,NaN,Boolean
5,ItalianFiscalCode,ItalianVAT,Error,Document,NaN,Plate,Address,NaN,ProvinceCode,Region,...,Name,NameSurname,NaN,NaN,String,EMail,Error,Error,BiologicalSex,Boolean
6,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Tax,Plate,NaN,ItalianZIPCode,Error,Region,...,Name,Error,SurnameName,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
7,NaN,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,Error,Region,...,Name,NameSurname,NaN,Surname,String,Error,NaN,NaN,BiologicalSex,Boolean
8,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,NaN,NaN,Error,...,Name,NaN,SurnameName,NaN,String,EMail,PhoneNumber,NaN,BiologicalSex,Boolean
9,ItalianFiscalCode,NaN,CadastreCode,Document,Tax,NaN,NaN,ItalianZIPCode,ProvinceCode,Region,...,NaN,NameSurname,SurnameName,Surname,String,EMail,PhoneNumber,NaN,NaN,Boolean


In [ ]:
model.predict_dataframe(X)

In [ ]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))